## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 5th Jan. 12th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-9-2025,Condo/Co-op,601 SW 141st Ave Unit 401P,Pembroke Pines,FL,33027.0,325000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/601-S...,MARMLS,A11717919,N,Y,26.002195,-80.332097


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-10    16
2025-01-06    16
2025-01-07    15
2025-01-08    15
2025-01-09    13
2025-01-13     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
65,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605
43,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,MARMLS,A11641255,N,Y,26.118835,-80.141476
24,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1800-S...,MARMLS,A11656807,N,Y,26.207610,-80.093358
30,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Beaches MLS,F10460484,N,Y,26.304202,-80.077943
71,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11551319,N,Y,25.989969,-80.118043
3,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10463762,N,Y,26.236771,-80.089682
33,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2715...,MARMLS,A11643148,N,Y,26.162622,-80.101460
61,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/500...,MARMLS,A11607975,N,Y,25.991623,-80.126240
2,PAST SALE,2025-01-07,Condo/Co-op,2501 S Ocean Dr #1122,Hollywood,FL,33019.0,515000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2501-S-Oce...,MARMLS,A11720209,N,Y,25.997180,-80.117628
32,PAST SALE,2025-01-10,Condo/Co-op,2301 S Ocean Dr Ph 02,Hollywood,FL,33019.0,490000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11678881,N,Y,25.997517,-80.116976


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,...,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None
1,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,...,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jonathan Howard,Douglas Elliman,Anthony Santangelo,Douglas Elliman,None,None,None,None
2,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,...,-80.093358,https://www.redfin.com/FL/Pompano-Beach/1800-S...,Norma Schneck,Douglas Elliman,None,None,None,None,None,None
3,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,...,-80.077943,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Nikolas Marulis,RE/MAX First,None,None,Joanna Jimenez,"Compass Florida, LLC.",None,None
4,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,...,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Carlos Suarez,Black Swan Home Group LLC,None,None,Mariana Tevez,EXP Realty LLC,None,None
5,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,...,-80.089682,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Patricia Davila,Century 21 Hansen Realty Inc,None,None,Patricia Davila,Century 21 Hansen Realty Inc,None,None
6,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,...,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/2715...,Alex McCoy,"Compass Florida, LLC",Amber Zakharia,"Compass Florida, LLC",None,None,None,None
7,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,...,-80.126240,https://www.redfin.com/FL/Hallandale-Beach/500...,Cindy Orlinsky,Douglas Elliman,None,None,Don Nelson,Coldwell Banker Realty,None,None
8,PAST SALE,2025-01-07,Condo/Co-op,2501 S Ocean Dr #1122,Hollywood,FL,33019.0,515000.0,1.0,1.0,...,-80.117628,https://www.redfin.com/FL/Hollywood/2501-S-Oce...,Svetlana Perez,"LP Global Realty, LLC",Ricardo Perez,"LP Global Realty, LLC",Svetlana Perez,"LP Global Realty, LLC",None,None
9,PAST SALE,2025-01-10,Condo/Co-op,2301 S Ocean Dr Ph 02,Hollywood,FL,33019.0,490000.0,1.0,1.5,...,-80.116976,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,Beata Hill,Optimar International Realty,None,None,Beata Hill,Optimar International Realty,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

76


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$326,342


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$281


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$24,802,000


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
100
Input Previous Week Condo Average Sales Price:
344588
Input Previous Week Condo Average PSF:
271
Input Previous Week Condo Sales total (ex: 198_000_000)
34_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange
1,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,100 Las Olas Condo,1867.0,NaN,2020.0,NaN,857.0,2178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,MARMLS,A11641255,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jonathan Howard,Douglas Elliman,Anthony Santangelo,Douglas Elliman,None,None,None,None,2,blue
2,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,HAMPTON BEACH CLUB CONDO,3340.0,NaN,1980.0,NaN,367.0,2623.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1800-S...,MARMLS,A11656807,N,Y,26.207610,-80.093358,https://www.redfin.com/FL/Pompano-Beach/1800-S...,Norma Schneck,Douglas Elliman,None,None,None,None,None,None,3,blue
3,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,Mediterranea Condo,2120.0,NaN,1999.0,NaN,429.0,1944.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Beaches MLS,F10460484,N,Y,26.304202,-80.077943,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Nikolas Marulis,RE/MAX First,None,None,Joanna Jimenez,"Compass Florida, LLC.",None,None,4,blue
4,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,SEA AIR TOWERS CONDO,1459.0,NaN,1971.0,NaN,425.0,795.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11551319,N,Y,25.989969,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Carlos Suarez,Black Swan Home Group LLC,None,None,Mariana Tevez,EXP Realty LLC,None,None,5,blue
5,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,Jamaica House Condominium,1200.0,NaN,1969.0,NaN,467.0,692.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10463762,N,Y,26.236771,-80.089682,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Patricia Davila,Century 21 Hansen Realty Inc,None,None,Patricia Davila,Century 21 Hansen Realty Inc,None,None,6,blue
6,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,EMBASSY TOWER II INC COND,1200.0,NaN,1973.0,NaN,446.0,1421.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2715...,MARMLS,A11643148,N,Y,26.162622,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/2715...,Alex McCoy,"Compass Florida, LLC",Amber Zakharia,"Compass Florida, LLC",None,None,None,None,7,blue
7,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,OLYMPUS CONDO PHASE,1780.0,NaN,1974.0,NaN,296.0,1265.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/500...,MARMLS,A11607975,N,Y,25.991623,-80.126240,https://www.redfin.com/FL/Hallandale-Beach/500...,Cindy Orlinsky,Douglas Elliman,None,None,Don Nelson,Coldwell Banker Realty,None,None,8,blue
8,PAST SALE,2025-01-07,Condo/Co-op,2501 S Ocean Dr #1122,Hollywood,FL,33019.0,515000.0,1.0,1.0,WAVE CONDO,745.0,NaN,1968.0,NaN,691.0,963.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2501-S-Oce...,MARMLS,A11720209,

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_01132024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'Aquablu at 920 Intracoastal Dr #1201 in Fort Lauderdale'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange,Aquablu at 920 Intracoastal Dr #1201 in Fort L...
1,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,100 Las Olas Condo,1867.0,NaN,2020.0,NaN,857.0,2178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,MARMLS,A11641255,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jonathan Howard,Douglas Elliman,Anthony Santangelo,Douglas Elliman,None,None,None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #380...
2,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,HAMPTON BEACH CLUB CONDO,3340.0,NaN,1980.0,NaN,367.0,2623.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1800-S...,MARMLS,A11656807,N,Y,26.207610,-80.093358,https://www.redfin.com/FL/Pompano-Beach/1800-S...,Norma Schneck,Douglas Elliman,None,None,None,None,None,None,3,blue,HAMPTON BEACH CLUB CONDO at 1800 S Ocean Blvd ...
3,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,Mediterranea Condo,2120.0,NaN,1999.0,NaN,429.0,1944.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Beaches MLS,F10460484,N,Y,26.304202,-80.077943,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Nikolas Marulis,RE/MAX First,None,None,Joanna Jimenez,"Compass Florida, LLC.",None,None,4,blue,Mediterranea Condo at 1228 Hillsboro Mile #102...
4,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,SEA AIR TOWERS CONDO,1459.0,NaN,1971.0,NaN,425.0,795.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11551319,N,Y,25.989969,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Carlos Suarez,Black Swan Home Group LLC,None,None,Mariana Tevez,EXP Realty LLC,None,None,5,blue,SEA AIR TOWERS CONDO at 3725 S Ocean Dr #924 i...
5,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,Jamaica House Condominium,1200.0,NaN,1969.0,NaN,467.0,692.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10463762,N,Y,26.236771,-80.089682,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Patricia Davila,Century 21 Hansen Realty Inc,None,None,Patricia Davila,Century 21 Hansen Realty Inc,None,None,6,blue,Jamaica House Condominium at 305 N Pompano Bea...
6,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,EMBASSY TOWER II INC COND,1200.0,NaN,1973.0,NaN,446.0,1421.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2715...,MARMLS,A11643148,N,Y,26.162622,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/2715...,Alex McCoy,"Compass Florida, LLC",Amber Zakharia,"Compass Florida, LLC",None,None,None,None,7,blue,EMBASSY TOWER II INC COND at 2715 N Ocean Blvd...
7,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,OLYMPUS CONDO PHASE,1780.0,NaN,1974.0,NaN,296.0,1265.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/500...,MARMLS,A11607975,N,Y,25.991623,-80.126240,h

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Aquablu closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $490,000 to $2,175,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Pompano Beach, Hillsboro Beach, Hallandale Beach, Hollywood

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $24,802,000 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 100 condo sales totaling $34,500,000.

Last week’s units sold for an average of $326,342, lower than the $344,588 average price from the previous

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange,Aquablu at 920 Intracoastal Dr #1201 in Fort L...
1,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,100 Las Olas Condo,1867.0,NaN,2020.0,NaN,857.0,2178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,MARMLS,A11641255,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jonathan Howard,Douglas Elliman,Anthony Santangelo,Douglas Elliman,None,None,None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #380...
2,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,HAMPTON BEACH CLUB CONDO,3340.0,NaN,1980.0,NaN,367.0,2623.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1800-S...,MARMLS,A11656807,N,Y,26.207610,-80.093358,https://www.redfin.com/FL/Pompano-Beach/1800-S...,Norma Schneck,Douglas Elliman,None,None,None,None,None,None,3,blue,HAMPTON BEACH CLUB CONDO at 1800 S Ocean Blvd ...
3,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,Mediterranea Condo,2120.0,NaN,1999.0,NaN,429.0,1944.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Beaches MLS,F10460484,N,Y,26.304202,-80.077943,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Nikolas Marulis,RE/MAX First,None,None,Joanna Jimenez,"Compass Florida, LLC.",None,None,4,blue,Mediterranea Condo at 1228 Hillsboro Mile #102...
4,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,SEA AIR TOWERS CONDO,1459.0,NaN,1971.0,NaN,425.0,795.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11551319,N,Y,25.989969,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Carlos Suarez,Black Swan Home Group LLC,None,None,Mariana Tevez,EXP Realty LLC,None,None,5,blue,SEA AIR TOWERS CONDO at 3725 S Ocean Dr #924 i...
5,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,Jamaica House Condominium,1200.0,NaN,1969.0,NaN,467.0,692.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10463762,N,Y,26.236771,-80.089682,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Patricia Davila,Century 21 Hansen Realty Inc,None,None,Patricia Davila,Century 21 Hansen Realty Inc,None,None,6,blue,Jamaica House Condominium at 305 N Pompano Bea...
6,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,EMBASSY TOWER II INC COND,1200.0,NaN,1973.0,NaN,446.0,1421.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2715...,MARMLS,A11643148,N,Y,26.162622,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/2715...,Alex McCoy,"Compass Florida, LLC",Amber Zakharia,"Compass Florida, LLC",None,None,None,None,7,blue,EMBASSY TOWER II INC COND at 2715 N Ocean Blvd...
7,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,OLYMPUS CONDO PHASE,1780.0,NaN,1974.0,NaN,296.0,1265.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/500...,MARMLS,A11607975,N,Y,25.991623,-80.126240,h

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Aquablu closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $490,000 to $2,175,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Lauderdale By The Sea, Fort Lauderdale, Pompano Beach, Hillsboro Beach, Hallandale Beach, Hollywood

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $24,802,000 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 100 condo sales totaling $34,500,000.

Last week’s units sold for an average of $326,342, lower than the $344,588 average price from the previous

In [59]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/920-Intracoastal-Dr-33304/unit-1201/home/79883228


In [61]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-3804/home/113846965


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hollywood/2301-S-Ocean-Dr-33019/unit-02/home/193211142


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hollywood/2501-S-Ocean-Dr-33019/unit-1122/home/42018573


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-3804/home/113846965


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hallandale-Beach/500-Three-Islands-Blvd-33009/unit-118/home/41956620


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-07,Condo/Co-op,920 Intracoastal Dr #1201,Fort Lauderdale,FL,33304.0,2175000.0,3.0,3.5,Aquablu,3195.0,NaN,2018.0,NaN,681.0,4161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10434063,N,Y,26.135735,-80.109605,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Ronald Lennen,"Compass Florida, LLC",Maureen Rotella,"Compass Florida, LLC",Jeremy Willard,RE/MAX Experience,None,None,1,orange,Aquablu at 920 Intracoastal Dr #1201 in Fort L...
1,PAST SALE,2025-01-08,Condo/Co-op,100 E Las Olas Blvd #3804,Fort Lauderdale,FL,33301.0,1600000.0,2.0,3.0,100 Las Olas Condo,1867.0,NaN,2020.0,NaN,857.0,2178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,MARMLS,A11641255,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jonathan Howard,Douglas Elliman,Anthony Santangelo,Douglas Elliman,None,None,None,None,2,blue,100 Las Olas Condo at 100 E Las Olas Blvd #380...
2,PAST SALE,2025-01-09,Condo/Co-op,1800 S Ocean Blvd #410,Lauderdale By The Sea,FL,33062.0,1225000.0,3.0,3.5,HAMPTON BEACH CLUB CONDO,3340.0,NaN,1980.0,NaN,367.0,2623.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1800-S...,MARMLS,A11656807,N,Y,26.207610,-80.093358,https://www.redfin.com/FL/Pompano-Beach/1800-S...,Norma Schneck,Douglas Elliman,None,None,None,None,None,None,3,blue,HAMPTON BEACH CLUB CONDO at 1800 S Ocean Blvd ...
3,PAST SALE,2025-01-09,Condo/Co-op,1228 Hillsboro Mile #102,Hillsboro Beach,FL,33062.0,910000.0,2.0,2.5,Mediterranea Condo,2120.0,NaN,1999.0,NaN,429.0,1944.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Beaches MLS,F10460484,N,Y,26.304202,-80.077943,https://www.redfin.com/FL/Hillsboro-Beach/1228...,Nikolas Marulis,RE/MAX First,None,None,Joanna Jimenez,"Compass Florida, LLC.",None,None,4,blue,Mediterranea Condo at 1228 Hillsboro Mile #102...
4,PAST SALE,2025-01-06,Condo/Co-op,3725 S Ocean Dr #924,Hollywood,FL,33019.0,620000.0,2.0,2.0,SEA AIR TOWERS CONDO,1459.0,NaN,1971.0,NaN,425.0,795.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11551319,N,Y,25.989969,-80.118043,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,Carlos Suarez,Black Swan Home Group LLC,None,None,Mariana Tevez,EXP Realty LLC,None,None,5,blue,SEA AIR TOWERS CONDO at 3725 S Ocean Dr #924 i...
5,PAST SALE,2025-01-08,Condo/Co-op,305 N Pompano Beach Blvd #1402,Pompano Beach,FL,33062.0,560000.0,2.0,2.0,Jamaica House Condominium,1200.0,NaN,1969.0,NaN,467.0,692.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10463762,N,Y,26.236771,-80.089682,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Patricia Davila,Century 21 Hansen Realty Inc,None,None,Patricia Davila,Century 21 Hansen Realty Inc,None,None,6,blue,Jamaica House Condominium at 305 N Pompano Bea...
6,PAST SALE,2025-01-07,Condo/Co-op,2715 N Ocean Blvd Unit 16A,Fort Lauderdale,FL,33308.0,535000.0,2.0,2.0,EMBASSY TOWER II INC COND,1200.0,NaN,1973.0,NaN,446.0,1421.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2715...,MARMLS,A11643148,N,Y,26.162622,-80.101460,https://www.redfin.com/FL/Fort-Lauderdale/2715...,Alex McCoy,"Compass Florida, LLC",Amber Zakharia,"Compass Florida, LLC",None,None,None,None,7,blue,EMBASSY TOWER II INC COND at 2715 N Ocean Blvd...
7,PAST SALE,2025-01-07,Condo/Co-op,500 Three Islands Blvd #118,Hallandale Beach,FL,33009.0,527500.0,3.0,2.0,OLYMPUS CONDO PHASE,1780.0,NaN,1974.0,NaN,296.0,1265.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/500...,MARMLS,A11607975,N,Y,25.991623,-80.126240,h

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 15) ## List (Earlier) date
date2 = datetime(2025, 1, 8) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

146


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['$/SQUARE FEET'].astype(float)

0    681.0
1    857.0
2    367.0
3    429.0
4    425.0
5    467.0
6    446.0
7    296.0
8    691.0
9    490.0
Name: $/SQUARE FEET, dtype: float64

In [70]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [72]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [73]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-07,920 Intracoastal Dr #1201 in Fort Lauderdale,"$2,175,000",3,3.5,"3,195",2018,$681,Beaches MLS,F10434063,"Ronald Lennen Compass Florida, LLC Maureen Rot...",Jeremy Willard RE/MAX Experience
1,2025-01-08,100 E Las Olas Blvd #3804 in Fort Lauderdale,"$1,600,000",2,3.0,"1,867",2020,$857,MARMLS,A11641255,Jonathan Howard Douglas Elliman Anthony Santan...,
2,2025-01-09,1800 S Ocean Blvd #410 in Lauderdale By The Sea,"$1,225,000",3,3.5,"3,340",1980,$367,MARMLS,A11656807,Norma Schneck Douglas Elliman,
3,2025-01-09,1228 Hillsboro Mile #102 in Hillsboro Beach,"$910,000",2,2.5,"2,120",1999,$429,Beaches MLS,F10460484,Nikolas Marulis RE/MAX First,"Joanna Jimenez Compass Florida, LLC."
4,2025-01-06,3725 S Ocean Dr #924 in Hollywood,"$620,000",2,2.0,"1,459",1971,$425,MARMLS,A11551319,Carlos Suarez Black Swan Home Group LLC,Mariana Tevez EXP Realty LLC
5,2025-01-08,305 N Pompano Beach Blvd #1402 in Pompano Beach,"$560,000",2,2.0,"1,200",1969,$467,Beaches MLS,F10463762,Patricia Davila Century 21 Hansen Realty Inc,Patricia Davila Century 21 Hansen Realty Inc
6,2025-01-07,2715 N Ocean Blvd Unit 16A in Fort Lauderdale,"$535,000",2,2.0,"1,200",1973,$446,MARMLS,A11643148,"Alex McCoy Compass Florida, LLC Amber Zakharia...",
7,2025-01-07,500 Three Islands Blvd #118 in Hallandale Beach,"$527,500",3,2.0,"1,780",1974,$296,MARMLS,A11607975,Cindy Orlinsky Douglas Elliman,Don Nelson Coldwell Banker Realty
8,2025-01-07,2501 S Ocean Dr #1122 in Hollywood,"$515,000",1,1.0,745,1968,$691,MARMLS,A11720209,"Svetlana Perez LP Global Realty, LLC Ricardo P...","Svetlana Perez LP Global Realty, LLC"
9,2025-01-10,2301 S Ocean Dr Ph 02 in Hollywood,"$490,000",1,1.5,"1,000",1978,$490,MARMLS,A11678881,Beata Hill Optimar International Realty,Beata Hill Optimar International Realty
